In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 5.1 MB/s 


In [ ]:
! pip install PyDrive

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1a1mZHqe3o1psMlsaXrqIUOe9IiYQ2gRT/view?usp=sharing'

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Dec.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Dec.csv')
df


,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative
...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral


In [ ]:
df['text'] = df['text'].astype('str') 

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [ ]:
df['text'] = clean_tweets(df['text'])

In [ ]:
df['text'].head()

0    "Tucker Carlson seem to have forgotten that Ru...
1    "Analysis: Would Putin get away with invading ...
2    "Kiev, Ukraine - Ukrainian defense officials s...
3    Russia-Ukraine tensions: why Russia is putting...
4     I don’t think it’s that deep. I think, becaus...
Name: text, dtype: object

In [ ]:
scores = []

# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for i in range(df['text'].shape[0]):
    #print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyser.polarity_scores(df['text'][i])["compound"]
    pos = analyser.polarity_scores(df['text'][i])["pos"]
    neu = analyser.polarity_scores(df['text'][i])["neu"]
    neg = analyser.polarity_scores(df['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [ ]:
df.head(10)

,Unnamed: 0,text,1,2,3,Label,Compound,Positive,Negative,Neutral
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,-0.2942,0.000,0.075,0.925
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,0.0000,0.000,0.000,1.000
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,0.3818,0.122,0.000,0.878
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,-0.4019,0.000,0.144,0.856
4,Q5,"I don’t think it’s that deep. I think, becaus...",0.1,0.0,0.9,negative,-0.6697,0.000,0.097,0.903
5,Q6,Is he selling the Ukraine to Russia for a com...,0.0,0.2,0.8,negative,0.0000,0.000,0.000,1.000
6,Q7,"""Russia positions more forces, supply lines as...",0.0,0.6,0.4,neutral,-0.4215,0.000,0.177,0.823
7,Q8,U.S. Intelligence Sees Russian Plan for Possib...,0.1,0.6,0.3,neutral,0.4767,0.279,0.000,0.721
8,Q9,President Joe Biden threatened to strike Russi...,0.2,0.5,0.3,neutral,-0.7184,0.000,0.333,0.667
9,Q10,"""Look who was Putin's puppet all along?\n""Admi...",0.1,0.3,0.6,negative,0.0000,0.000,0.000,1.000


In [ ]:
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('positive')
    elif i <= -0.05 :
        sentiment.append('negative')
    else:
        sentiment.append('neutral')
df["Sentiment"] = sentiment
df.head()

,Unnamed: 0,text,1,2,3,Label,Compound,Positive,Negative,Neutral,Sentiment
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,-0.2942,0.000,0.075,0.925,negative
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,0.0000,0.000,0.000,1.000,neutral
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,0.3818,0.122,0.000,0.878,positive
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,-0.4019,0.000,0.144,0.856,negative
4,Q5,"I don’t think it’s that deep. I think, becaus...",0.1,0.0,0.9,negative,-0.6697,0.000,0.097,0.903,negative


In [ ]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [ ]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})
df = df.rename(columns={"Sentiment":"Vader Label"})

In [ ]:
df['Vader Label'].value_counts()

negative    53
positive    26
neutral     21
Name: Vader Label, dtype: int64

In [ ]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Vader = balanced_accuracy_score(df['Manual Label'], df['Vader Label'])
 p_r_f1_class_Vader = precision_recall_fscore_support(df['Manual Label'], df['Vader Label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Vader = precision_recall_fscore_support(df['Manual Label'], df['Vader Label'], average='macro')

In [ ]:
#0.42 accuracy with pre-processing step. check what it is without
print(b_a_Vader)
print(p_r_f1_Vader)
print(p_r_f1_class_Vader)

0.3878917378917379
(0.42, 0.42, 0.41999999999999993, None)
(array([0.50943396, 0.66666667, 0.03846154]), array([0.51923077, 0.31111111, 0.33333333]), array([0.51428571, 0.42424242, 0.06896552]), array([52, 45,  3]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['Vader Label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

Vader Label,negative,neutral,positive
Manual Label,,,
negative,27,7,18
neutral,24,14,7
positive,2,0,1


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['Vader Label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.51923077, 0.31111111, 0.33333333])

In [ ]:
matrix

array([[27,  7, 18],
       [24, 14,  7],
       [ 2,  0,  1]])

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('VADER_Test_Dec.csv')
!cp VADER_Test_Dec.csv "/content/drive/My Drive/Colab Notebooks"

